## 第一步：环境配置与数据读取

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
# import open3d as o3d  # 推荐安装 open3d 用于3D可视化

# 配置路径
DATA_ROOT = "/home/xzy/datasets/Rellis-3D" 
SEQ = "00004"
FRAME_ID = "000000"  # 示例帧

# 路径拼接 (根据 Rellis 目录结构)
# Rellis 通常结构: 00004/os1_cloud_node_kitti_bin/000000.bin
bin_path = os.path.join(DATA_ROOT, SEQ, 'os1_cloud_node_kitti_bin', f'{FRAME_ID}.bin')
img_path = os.path.join(DATA_ROOT, SEQ, 'pylon_camera_node', f'frame{FRAME_ID}-...jpg') # 需补全实际文件名

# 读取点云 (x, y, z, intensity)
points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
print(f"Point cloud shape: {points.shape}")

Point cloud shape: (131072, 4)


## 第二步：理解相机与雷达的关系（外参标定）

## 第三步：解析 poses.txt 与世界坐标系

坐标轴位姿可视化：红绿蓝 x y z

使用 o3d.geometry.TriangleMesh.create_sphere 在轨迹两端放置了球体。

起点：绿色 [0, 1, 0]，代表 "Go/Start"。

终点：红色 [1, 0, 0]，代表 "Stop/End"。

poses.txt立即为雷达系位姿（前x左y上z），可视化发现车子是倒开。

In [2]:
import open3d as o3d
import numpy as np



def parse_poses(filename):
    poses = []
    with open(filename, 'r') as f:
        for line in f:
            values = [float(v) for v in line.strip().split()]
            pose = np.eye(4)
            pose[:3, :4] = np.array(values).reshape(3, 4)
            poses.append(pose)
    return poses

pose_file = os.path.join(DATA_ROOT, SEQ, 'poses.txt')
poses = parse_poses(pose_file)

def visualize_trajectory_open3d(poses, visual_interval=10):
    """
    poses: 4x4 变换矩阵列表
    visual_interval: 每隔多少帧画一个坐标轴 (避免太密集)
    """
    geometries = []

    # 1. 创建轨迹线 (LineSet)
    # 提取所有平移向量 (位置)
    points = [p[:3, 3] for p in poses]
    
    # 定义线段连接关系: 0-1, 1-2, 2-3 ...
    lines = [[i, i+1] for i in range(len(points)-1)]
    
    # 创建 LineSet 对象
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.paint_uniform_color([0, 0, 0]) # 黑色轨迹线
    geometries.append(line_set)

    # 2. 创建坐标轴 (Coordinate Frames)
    # 红色=X(前), 绿色=Y(左), 蓝色=Z(上)
    for i in range(0, len(poses), visual_interval):
        # 创建一个小坐标轴，size 控制大小
        mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2.0, origin=[0, 0, 0])
        
        # 将坐标轴变换到对应的位姿位置
        mesh_frame.transform(poses[i])
        geometries.append(mesh_frame)

    # 3. 特殊处理：起点 (Start Node)
    # 绿色球体 + 较大的坐标轴
    start_pose = poses[0]
    start_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.8)
    start_sphere.paint_uniform_color([0, 1, 0])  # 纯绿色
    start_sphere.translate(start_pose[:3, 3])
    
    start_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3.0, origin=[0, 0, 0])
    start_frame.transform(start_pose)
    
    geometries.extend([start_sphere, start_frame])

    # 4. 特殊处理：终点 (End Node)
    # 红色球体 + 较大的坐标轴
    end_pose = poses[-1]
    end_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.8)
    end_sphere.paint_uniform_color([1, 0, 0])    # 纯红色
    end_sphere.translate(end_pose[:3, 3])
    
    end_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=3.0, origin=[0, 0, 0])
    end_frame.transform(end_pose)
    
    geometries.extend([end_sphere, end_frame])

    # 5. 启动可视化
    print("Green sphere: START | Red sphere: END")
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="Trajectory Viewer (Start=Green, End=Red)", width=1280, height=720)
    
    for g in geometries:
        vis.add_geometry(g)
    
    # 获取渲染控制并设置点的大小（如果需要的话）
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0.1, 0.1, 0.1]) # 设置深色背景更易观察坐标轴
    opt.line_width = 2.0
    
    vis.run()
    vis.destroy_window()

visualize_trajectory_open3d(poses, visual_interval=10)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Green sphere: START | Red sphere: END


## 第四步：点云建图（Point Cloud Mapping）

绿球为起始点，红球为终点，进一步验证小车倒退。

红绿蓝XYZ。

确定poses.txt就是雷达系，因为运动过程中Z轴保持不变。成功将点云拼接与位姿配合使用。

In [3]:
import os
import numpy as np
import open3d as o3d

# ================= 1. 配置参数 =================
# 请确保 DATA_ROOT 和 SEQ 已定义，或者在此处重新定义
# DATA_ROOT = "/home/xzy/datasets/Rellis-3D" 
# SEQ = "00004"
LIDAR_FOLDER = 'os1_cloud_node_kitti_bin'

# [参数设置] 帧间隔
STEP = 50  
# [参数设置] 最大帧数
MAX_FRAMES = 200 

# ================= 2. 辅助函数 =================

def load_lidar_col(bin_path):
    """ 读取点云并转换为齐次坐标列向量 (4, N) """
    # Rellis/Kitti bin 是 float32, x,y,z,intensity
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
    xyz = points[:, :3]
    num_points = xyz.shape[0]
    ones = np.ones((num_points, 1))
    points_h = np.hstack([xyz, ones])
    return points_h.T

def parse_poses(filename):
    """ 读取 poses.txt """
    poses = []
    if not os.path.exists(filename):
        print(f"Warning: {filename} not found.")
        return []
    with open(filename, 'r') as f:
        for line in f:
            values = [float(v) for v in line.strip().split()]
            pose = np.eye(4)
            pose[:3, :4] = np.array(values).reshape(3, 4)
            poses.append(pose)
    return poses

def get_custom_colors(world_points_col):
    """
    根据 [世界坐标] (x,y,z) 的象限/范围设定颜色
    输入: world_points_col (4, N)
    输出: colors (3, N)
    """
    x = world_points_col[0, :]
    y = world_points_col[1, :]
    z = world_points_col[2, :]
    
    N = world_points_col.shape[1]
    
    # 默认背景色：使用黑色，这样彩色点更明显
    colors = np.zeros((3, N), dtype=np.float64) 
    
    # 颜色定义
    color_x = np.array([[1.0], [0.0], [0.0]]) # 红
    color_y = np.array([[0.0], [1.0], [0.0]]) # 绿
    color_z = np.array([[0.0], [0.0], [1.0]]) # 蓝
    
    # --- 你的自定义逻辑 (基于世界系坐标) ---
    
    # 1. World X>0, Y<0, Z<0 -> 红色
    mask_1 = (x > 0) & (y < 0) & (z < 0)
    colors[:, mask_1] = color_x

    # 2. World X<0, Y>0, Z<0 -> 绿色
    mask_2 = (x < 0) & (y > 0) & (z < 0)
    colors[:, mask_2] = color_y
    
    # 3. World X<0, Y<0, Z>0 -> 蓝色
    mask_3 = (x < 0) & (y < 0) & (z > 0)
    colors[:, mask_3] = color_z
    
    # 其余保持黑色
    
    return colors

def create_world_origin_visuals(size=10.0, plane_size=50.0):
    """
    创建世界系原点的可视化增强组件
    1. 粗壮的 XYZ 轴
    2. 巨大的淡黄色正交平面
    """
    geoms = []
    
    # --- A. 粗壮坐标轴 (使用圆柱体模拟) ---
    radius = size * 0.05 # 轴的粗细
    
    # X轴 (红)
    x_cyl = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=size)
    x_cyl.paint_uniform_color([1, 0, 0])
    # 默认圆柱是沿Z轴的，需要旋转+平移
    # 旋转: Z -> X (绕Y轴转90度)
    R_x = o3d.geometry.get_rotation_matrix_from_xyz([0, np.pi/2, 0])
    x_cyl.rotate(R_x, center=[0,0,0])
    x_cyl.translate([size/2, 0, 0]) # 移动使得一端在原点
    
    # Y轴 (绿)
    y_cyl = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=size)
    y_cyl.paint_uniform_color([0, 1, 0])
    # 旋转: Z -> Y (绕X轴转-90度)
    R_y = o3d.geometry.get_rotation_matrix_from_xyz([-np.pi/2, 0, 0])
    y_cyl.rotate(R_y, center=[0,0,0])
    y_cyl.translate([0, size/2, 0])
    
    # Z轴 (蓝)
    z_cyl = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=size)
    z_cyl.paint_uniform_color([0, 0, 1])
    z_cyl.translate([0, 0, size/2]) # Z轴本身就朝上，只需向上平移一半高度
    
    geoms.extend([x_cyl, y_cyl, z_cyl])
    
    # --- B. 正交平面 (淡黄色，半透明) ---
    # Open3D 简单可视化不支持完美的 alpha 透明度，但我们可以设置一种淡色网格或薄板
    # 这里使用很薄的长方体来代表平面
    
    plane_color = [1.0, 1.0, 0.8] # 淡黄色
    thickness = 0.05
    
    # XY Plane
    xy_plane = o3d.geometry.TriangleMesh.create_box(width=plane_size*2, height=plane_size*2, depth=thickness)
    xy_plane.paint_uniform_color(plane_color)
    xy_plane.translate([-plane_size, -plane_size, -thickness/2]) # 居中
    
    # XZ Plane
    xz_plane = o3d.geometry.TriangleMesh.create_box(width=plane_size*2, height=thickness, depth=plane_size*2)
    xz_plane.paint_uniform_color(plane_color)
    xz_plane.translate([-plane_size, -thickness/2, -plane_size])
    
    # YZ Plane
    yz_plane = o3d.geometry.TriangleMesh.create_box(width=thickness, height=plane_size*2, depth=plane_size*2)
    yz_plane.paint_uniform_color(plane_color)
    yz_plane.translate([-thickness/2, -plane_size, -plane_size])
    
    geoms.extend([xy_plane, xz_plane, yz_plane])
    
    return geoms

# ================= 3. 主逻辑 =================

pose_file = os.path.join(DATA_ROOT, SEQ, 'poses.txt')
lidar_dir = os.path.join(DATA_ROOT, SEQ, LIDAR_FOLDER)
poses = parse_poses(pose_file)

global_points_list = []
global_colors_list = []
visual_geometries = []

# --- 1. 添加世界系原点可视化 ---
print("Creating World Origin visualization...")
world_visuals = create_world_origin_visuals(size=5.0, plane_size=50.0)
visual_geometries.extend(world_visuals)

# --- 2. 遍历帧进行建图 ---
num_poses = len(poses)
limit = num_poses if MAX_FRAMES is None else min(num_poses, MAX_FRAMES)
frame_indices = range(0, limit, STEP)

print(f"Starting mapping (Frames: {limit}, Step: {STEP})...")

for i in frame_indices:
    T = poses[i] # World <- Lidar
    
    bin_path = os.path.join(lidar_dir, f"{i:06d}.bin")
    if not os.path.exists(bin_path): continue
    
    # A. 读取局部点云
    P_local = load_lidar_col(bin_path) 
    
    # B. 变换到世界系
    # P_world 是该点云在绝对世界系下的真实位置
    P_world = T @ P_local 
    
    # C. 基于【世界系坐标】计算颜色
    # 这能反映点云在绝对空间中的分布象限
    C_world = get_custom_colors(P_world)
    
    # D. 存储
    global_points_list.append(P_world[:3, :])
    global_colors_list.append(C_world)
    
    # E. 轨迹上的小坐标轴 (辅助)
    axis_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0)
    axis_mesh.transform(T)
    visual_geometries.append(axis_mesh)

# --- 3. 起终点特殊标注 ---
if len(poses) > 0:
    # 起点
    start_pose = poses[0]
    start_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
    start_sphere.paint_uniform_color([0, 1, 0]) # 绿球
    start_sphere.translate(start_pose[:3, 3])
    start_sphere.compute_vertex_normals()
    visual_geometries.append(start_sphere)
    
    # 终点
    last_idx = frame_indices[-1]
    end_pose = poses[last_idx]
    end_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
    end_sphere.paint_uniform_color([1, 0, 0]) # 红球
    end_sphere.translate(end_pose[:3, 3])
    end_sphere.compute_vertex_normals()
    visual_geometries.append(end_sphere)

# --- 4. 显示 ---
if global_points_list:
    all_points = np.hstack(global_points_list).T
    all_colors = np.hstack(global_colors_list).T
    
    print(f"Building PointCloud with {all_points.shape[0]} points...")
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(all_points)
    pcd.colors = o3d.utility.Vector3dVector(all_colors)
    
    # 这里的体素降采样可以根据电脑性能开启
    # pcd = pcd.voxel_down_sample(voxel_size=0.2)
    
    visual_geometries.append(pcd)
    
    print("Visualizing...")
    print("  - Thick Axis: World Origin (0,0,0)")
    print("  - Yellow Planes: World Orthogonal Planes")
    print("  - Green Sphere: Start Pose")
    print("  - Red Sphere: End Pose")
    
    o3d.visualization.draw_geometries(visual_geometries,
                                      window_name="World Frame Colored Map",
                                      width=1024, height=768)
else:
    print("No data found.")

Creating World Origin visualization...
Starting mapping (Frames: 200, Step: 50)...
Building PointCloud with 524288 points...
Visualizing...
  - Thick Axis: World Origin (0,0,0)
  - Yellow Planes: World Orthogonal Planes
  - Green Sphere: Start Pose
  - Red Sphere: End Pose


## 第五步：可视化与点云优化（可选）